In [1]:
# pre defined casting weight graph
# see more data/movie-actor/make_casting_graph.py

import pickle
from pprint import pprint 

with open('data/movie-actor/casting_graph.pkl', 'rb') as f:
    graph = pickle.load(f)

# casting weight of movie = 10001
pprint(sorted(graph['10001'].items(), key=lambda x:-x[1]))

[('4374', 0.1711229946524064),
 ('178', 0.15040106951871657),
 ('3241', 0.13101604278074866),
 ('47952', 0.11296791443850267),
 ('47953', 0.0962566844919786),
 ('19538', 0.08088235294117647),
 ('18991', 0.06684491978609626),
 ('47954', 0.05414438502673797),
 ('6038', 0.0427807486631016),
 ('24102', 0.032754010695187165),
 ('47955', 0.02406417112299465),
 ('16903', 0.016711229946524065),
 ('47956', 0.0106951871657754),
 ('47957', 0.006016042780748663),
 ('47958', 0.00267379679144385),
 ('47959', 0.0006684491978609625)]


In [2]:
# create idx to actor name function
with open('data/movie-actor/actors.csv', encoding='utf-8') as f:
    next(f)
    docs = [line[:-1].split('\t') for line in f]
    # English name if exist else Korean name
    _idx2actor = {doc[0]:(doc[2] if doc[2] else doc[1]) for doc in docs}

idx2actor = lambda idx: _idx2actor.get(idx, 'Unknown')

# create idx to movie name function
def append_year_countries(year, countries):
    if year and countries:
        return ' ({}, {})'.format(year, countries)
    elif year:
        return ' ({})'.format(year)
    elif countries:
        return ' ({})'.format(countries)
    return ''

with open('data/movie-actor/movies.csv', encoding='utf-8') as f:
    next(f)
    docs = [line[:-1].split('\t') for line in f]
    _idx2movie = {doc[0]:'{}{}'.format(doc[1], append_year_countries(doc[4], doc[5])) for doc in docs if len(docs)}
    _idx2year = {doc[0]:(int(doc[4]) if doc[4] else 0) for doc in docs if len(docs)}

idx2movie = lambda idx: _idx2movie.get(idx, 'Unknown')
idx2year = lambda idx: _idx2year.get(idx, 0)

# create idx to num comments
with open('data/movie-actor/num_comments.txt', encoding='utf-8') as f:
    docs = [line[:-1].split('\t') for line in f]
    _idx2numcomments = {movie_idx:int(num) for movie_idx, num in docs}
idx2numcomments = lambda idx: _idx2numcomments.get(idx,0)

In [3]:
idx2numcomments('10001')

3497

In [4]:
import sys
sys.path.append('data/movie-actor/')

from make_casting_graph import oneway_to_bidirected_graph
g = oneway_to_bidirected_graph(graph)

In [5]:
from pagerank import pagerank

rank = pagerank(g,
                bias=None,
                df=0.15,
                max_iter=30,
                converge_error=0.001,
                verbose=1)

Iteration = 1, diff = 0.7663714375352599, sum = 1.0000000000000322
Iteration = 2, diff = 0.5973778498022658, sum = 1.0000000000001374
Iteration = 3, diff = 0.47537799040888584, sum = 1.0000000000001266
Iteration = 4, diff = 0.3876545641819893, sum = 1.0000000000001508
Iteration = 5, diff = 0.32019959679781, sum = 1.0000000000000355
Iteration = 6, diff = 0.26685630373296054, sum = 1.0000000000000346
Iteration = 7, diff = 0.2236944600746007, sum = 0.9999999999999887
Iteration = 8, diff = 0.18813011706127114, sum = 1.000000000000185
Iteration = 9, diff = 0.15857142935860943, sum = 1.000000000000044
Iteration = 10, diff = 0.13385368096602926, sum = 1.0000000000000162
Iteration = 11, diff = 0.11311713940222566, sum = 1.0000000000000244
Iteration = 12, diff = 0.09567336023364649, sum = 1.0000000000000602
Iteration = 13, diff = 0.08097322685291314, sum = 0.9999999999999941
Iteration = 14, diff = 0.06857008757046891, sum = 1.0000000000000024
Iteration = 15, diff = 0.05809138788591365, sum = 1.

In [6]:
# top rank movie
# filtering Korean movie
movie_rank = {node:rank for node, rank in rank.items() if node[0] == 'm'}
actor_rank = {node:rank for node, rank in rank.items() if node[0] == 'a'}

korean_movies = {movie:weight for movie, weight in movie_rank.items() if '한국)' in idx2movie(movie.split()[1])}

# movies starring many famous actors
for movie, _ in sorted(korean_movies.items(), key=lambda x:-x[1])[:100]:
    movie_idx = movie.split()[1]
    print(idx2movie(movie_idx), idx2year(movie_idx))

개그콘서트 (1999, 한국) 1999
폭소 클럽 (한국) 0
깡패와 건달로 본 한국 100년 (한국) 0
티라노의 발톱 (1994, 한국) 1994
살인의 강 (2010, 한국) 2010
신세대 보고서 어른들은 몰라요 (한국) 0
박하사탕 (2000, 한국) 2000
기담전설 2 - 소름 (2009, 한국) 2009
나와 함께 블루스를 (한국) 0
시간의 춤 (2009, 한국) 2009
우리 학교 (2007, 한국) 2007
그 사람 추기경 (2014, 한국) 2014
별별 이야기 2 - 여섯 빛깔 무지개 (2008, 한국) 2008
목포는 항구다 (2004, 한국) 2004
실미도 (2003, 한국) 2003
웃찾사 (한국) 0
레볼루션 (한국) 0
원 나잇 스탠드 (2010, 한국) 2010
전설의 고향 1997 (한국) 0
우린 액션배우다 (2008, 한국) 2008
반드시 크게 들을 것 (2010, 한국) 2010
누구에게나 찬란한 (2014, 한국) 2014
이리 (2008, 한국) 2008
아 유 레디? (2002, 한국) 2002
마이너클럽 (한국) 0
코미디에 빠지다 (2012, 한국) 2012
신혼여행 (2000, 한국) 2000
JSA 남북공동초등학교 (한국) 0
봉자 (2000, 한국) 2000
작은 연못 (2010, 한국) 2010
자명고 (2009, 한국) 2009
토지 (2004, 한국) 2004
주몽 (2006, 한국) 2006
서울역 (2016, 한국) 2016
코미디쇼 웃으면 복이 와요 (한국) 0
진짜 사나이 (1996, 한국) 1996
섹스 볼란티어 (2010, 한국) 2010
얼짱시대_시즌6 (한국) 0
자카르타 (2000, 한국) 2000
이카루스 (한국) 0
쌀 (한국) 0
똥례... 하면 죽는다 (2015, 한국) 2015
모범생 (한국) 0
전설의 고향 1996 (한국) 0
다방의 푸른 꿈 (2017, 한국) 2017
바세코의 아이들 (2014, 한국) 2014
할머니의 먼 집 (2016, 한국) 2016
유관순 (

In [7]:
# prefer latest movies
bias = {node:(max(0, (idx2year(node.split()[1])-1990)) ** 2 if node[:5] == 'movie' else 0) for node in g}
_bias_sum = sum(bias.values())
bias = {k:v/_bias_sum for k,v in bias.items()}

In [8]:
rank = pagerank(g,
                bias=bias,
                df=0.15,
                max_iter=30,
                converge_error=0.001,
                verbose=1)

Iteration = 1, diff = 0.969773556763698, sum = 0.9999999999998367
Iteration = 2, diff = 0.7493001556628113, sum = 0.9999999999976444
Iteration = 3, diff = 0.606216589548318, sum = 1.0000000000001914
Iteration = 4, diff = 0.5000460599604785, sum = 0.9999999999994763
Iteration = 5, diff = 0.4175456505200614, sum = 1.0000000000000004
Iteration = 6, diff = 0.35060472392895137, sum = 0.9999999999997002
Iteration = 7, diff = 0.2955612726223386, sum = 1.000000000000051
Iteration = 8, diff = 0.24964992878198866, sum = 1.0000000000003182
Iteration = 9, diff = 0.21113697954459815, sum = 0.9999999999999318
Iteration = 10, diff = 0.17874010845539196, sum = 0.9999999999996855
Iteration = 11, diff = 0.1513938891636431, sum = 1.0000000000001346
Iteration = 12, diff = 0.12830732036698478, sum = 1.000000000000129
Iteration = 13, diff = 0.10877481984026531, sum = 1.0000000000002007
Iteration = 14, diff = 0.09224670949960435, sum = 1.0000000000002254
Iteration = 15, diff = 0.07824947533704155, sum = 0.99

In [9]:
# top rank movie
# filtering Korean movie
movie_rank = {node:rank for node, rank in rank.items() if node[0] == 'm'}
actor_rank = {node:rank for node, rank in rank.items() if node[0] == 'a'}

korean_movies = {movie:weight for movie, weight in movie_rank.items() if '한국)' in idx2movie(movie.split()[1])}

# movies starring many famous actors
for movie, _ in sorted(korean_movies.items(), key=lambda x:-x[1])[:100]:
    movie_idx = movie.split()[1]
    print(idx2movie(movie_idx), idx2year(movie_idx))

개그콘서트 (1999, 한국) 1999
SMTOWN THE STAGE (2015, 한국) 2015
판도라 (2016, 한국) 2016
연평해전 (2015, 한국) 2015
패션왕 코리아 (2013, 한국) 2013
나는 가수다 Ⅱ (2012, 한국) 2012
토요일이 좋다-오! 마이 베이비 (2014, 한국) 2014
히야 (2016, 한국) 2016
인천상륙작전 (2016, 한국) 2016
언프리티 랩스타 2 (2015, 한국) 2015
손님 (2015, 한국) 2015
흔들리는 물결 (2016, 한국) 2016
매시 업! (2015, 한국) 2015
학교 2013 (2012, 한국) 2012
올레 (2016, 한국) 2016
마이 리틀 텔레비전 (2015, 한국) 2015
패션왕 코리아 2 (2014, 한국) 2014
굿바이 그리고 헬로우 (2015, 한국) 2015
바흐를 꿈꾸며 언제나 칸타레 (2014, 한국) 2014
TOP 밴드 2 (2012, 한국) 2012
기담전설 2 - 소름 (2009, 한국) 2009
NO.MERCY (2014, 한국) 2014
내 친구의 집은 어디인가 (2015, 한국) 2015
여왕의 교실 (2013, 한국) 2013
슈퍼스타K 2 (2010, 한국) 2010
듀엣가요제 8+ (2015, 한국) 2015
드림나이트 (2015, 한국) 2015
자명고 (2009, 한국) 2009
미션: 톱스타를 훔쳐라 (2015, 한국) 2015
투유 프로젝트 - 슈가맨 (2015, 한국) 2015
탐정 : 더 비기닝 (2015, 한국) 2015
하이스쿨 : 러브온 (2014, 한국) 2014
용감한 작가들 (2014, 한국) 2014
다른 길이 있다 (2017, 한국) 2017
서울역 (2016, 한국) 2016
작은 연못 (2010, 한국) 2010
이웃집 좀비 (2010, 한국) 2010
태양의 후예 (2016, 한국) 2016
패션킬라 (2013, 한국) 2013
코미디빅리그 (2012, 한국) 2012
더 지니어스: 블랙 가넷 (

In [10]:
bias = {node:(max(0, (idx2numcomments(node.split()[1]))) if node[:5] == 'movie' else 0) for node in g}
_bias_sum = sum(bias.values())
bias = {k:v/_bias_sum for k,v in bias.items()}

rank = pagerank(g,
                bias=bias,
                df=0.15,
                max_iter=30,
                converge_error=0.001,
                verbose=1)

Iteration = 1, diff = 0.973522739945992, sum = 0.9999999999998503
Iteration = 2, diff = 0.7586092188109472, sum = 0.9999999999989364
Iteration = 3, diff = 0.612136395804904, sum = 1.0000000000001412
Iteration = 4, diff = 0.5041426449280897, sum = 0.9999999999997006
Iteration = 5, diff = 0.4205911434661914, sum = 1.000000000000023
Iteration = 6, diff = 0.35282857845348214, sum = 0.9999999999997904
Iteration = 7, diff = 0.29719895865328416, sum = 1.0000000000000666
Iteration = 8, diff = 0.25081504214011485, sum = 1.0000000000002365
Iteration = 9, diff = 0.21200677864225326, sum = 0.999999999999931
Iteration = 10, diff = 0.17937385549498275, sum = 0.9999999999997873
Iteration = 11, diff = 0.15187850974117437, sum = 1.0000000000000888
Iteration = 12, diff = 0.1286725921232069, sum = 1.0000000000001033
Iteration = 13, diff = 0.10905744233913678, sum = 1.000000000000183
Iteration = 14, diff = 0.09246572410873403, sum = 1.000000000000171
Iteration = 15, diff = 0.07842074920691855, sum = 0.999

In [11]:
# top rank movie
# filtering Korean movie
movie_rank = {node:rank for node, rank in rank.items() if node[0] == 'm'}
actor_rank = {node:rank for node, rank in rank.items() if node[0] == 'a'}

korean_movies = {movie:weight for movie, weight in movie_rank.items() if '한국)' in idx2movie(movie.split()[1])}

# movies starring many famous actors
for movie, _ in sorted(korean_movies.items(), key=lambda x:-x[1])[:100]:
    movie_idx = movie.split()[1]
    print(idx2movie(movie_idx), idx2year(movie_idx))

인천상륙작전 (2016, 한국) 2016
연평해전 (2015, 한국) 2015
국가대표 (2009, 한국) 2009
설국열차 (2013, 한국) 2013
국제시장 (2014, 한국) 2014
암살 (2015, 한국) 2015
7번방의 선물 (2013, 한국) 2013
곡성(哭聲) (2016, 한국) 2016
귀향 (2016, 한국) 2016
26년 (2012, 한국) 2012
부산행 (2016, 한국) 2016
아가씨 (2016, 한국) 2016
밀정 (2016, 한국) 2016
럭키 (2016, 한국) 2016
디 워 (2007, 한국) 2007
판도라 (2016, 한국) 2016
미스터 고 (2013, 한국) 2013
다세포 소녀 (2006, 한국) 2006
해운대 (2009, 한국) 2009
님아, 그 강을 건너지 마오 (2014, 한국) 2014
광해, 왕이 된 남자 (2012, 한국) 2012
수상한 그녀 (2014, 한국) 2014
괴물 (2006, 한국) 2006
아수라 (2016, 한국) 2016
역린 (2014, 한국) 2014
포화 속으로 (2010, 한국) 2010
검은 사제들 (2015, 한국) 2015
베테랑 (2015, 한국) 2015
덕혜옹주 (2016, 한국) 2016
감기 (2013, 한국) 2013
타워 (2012, 한국) 2012
늑대소년 (2012, 한국) 2012
왕의 남자 (2005, 한국) 2005
라디오 스타 (2006, 한국) 2006
과속스캔들 (2008, 한국) 2008
쎄시봉 (2015, 한국) 2015
미녀는 괴로워 (2006, 한국) 2006
다이빙벨 (2014, 한국) 2014
신세계 (2013, 한국) 2013
세븐 데이즈 (2007, 한국) 2007
천안함 프로젝트 (2013, 한국) 2013
아저씨 (2010, 한국) 2010
연가시 (2012, 한국) 2012
검사외전 (2016, 한국) 2016
숨바꼭질 (2013, 한국) 2013
킹콩을 들다 (2009, 한국) 2009
7급 공무원 (2009,

In [12]:
# top actor
for actor, _ in sorted(actor_rank.items(), key=lambda x:-x[1])[:100]:
    actor_idx = actor.split()[1]
    actor_name = idx2actor(actor_idx)
    if actor_name != actor_name.lower():
        continue
    print(actor_name, end=' / ')

황하나 / 김도윤 / 윤제리 / 이정열 / 주걸륜 / 서승인 / 조주경 / 남호정 / 강동원 / 이동호 / 박태주 / 홍원기 / 김강우 / 유민채 / 조은형 / 김미경 / 김윤석 / 왕석현 / 공유 / 권예영 / 유호정 / 이자스민 / 염현서 / 황인준 / 유해진 / 박수일 / 김도진 / 김중기 / 송서하 / 오달수 / 서동수 / 황정민 / 조정석 / 김의성 / 이승준 / 박형우 / 박로사 / 안소희 / 박영수 / 한재영 / 김예준 / 베니 / 엄홍식 / 한동규 / 김창회 / 김주완 / 구본웅 / 김비비 / 나광훈 / 신정근 / 장윤주 / 윤빈 / 이예은 / 김수안 / 아부다드 / 이라혜 / 정기섭 / 박희정 / 김환희 / 김하나 / 김옥빈 / 정우 / 민복기 / 이현 / 

In [13]:
# user history
history = [
    121922, # 사도
    137952, # 밀정
    101901, # 변호인
    93728, # 관상
    62328 # 설국열차
]

history_set = {'movie {}'.format(movie_idx) for movie_idx in history}

bias = {node:1 if node in history_set else 0 for node in g}
_bias_sum = sum(bias.values())
bias = {k:v/_bias_sum for k,v in bias.items()}

rank = pagerank(g,
                bias=bias,
                df=0.15,
                max_iter=30,
                converge_error=0.001,
                verbose=1)

Iteration = 1, diff = 0.9939953902834746, sum = 0.999999999999831
Iteration = 2, diff = 0.7897381309230553, sum = 0.9999999999963145
Iteration = 3, diff = 0.6346623248605532, sum = 1.0000000000002343
Iteration = 4, diff = 0.5190921295737689, sum = 0.9999999999964461
Iteration = 5, diff = 0.4301530296514967, sum = 1.0000000000000049
Iteration = 6, diff = 0.35923612519966064, sum = 0.9999999999986623
Iteration = 7, diff = 0.30174884021014736, sum = 1.0000000000000748
Iteration = 8, diff = 0.254140986691011, sum = 1.0000000000004101
Iteration = 9, diff = 0.2144913737387693, sum = 0.9999999999999244
Iteration = 10, diff = 0.18124793427046082, sum = 0.9999999999996377
Iteration = 11, diff = 0.1533006623214554, sum = 1.0000000000001834
Iteration = 12, diff = 0.1297480285465358, sum = 1.0000000000001124
Iteration = 13, diff = 0.10988230452173914, sum = 1.000000000000241
Iteration = 14, diff = 0.09309169588161727, sum = 1.0000000000002491
Iteration = 15, diff = 0.07890333681623005, sum = 0.999

In [14]:
# top rank movie
# filtering Korean movie
movie_rank = {node:rank for node, rank in rank.items() if node[0] == 'm'}
actor_rank = {node:rank for node, rank in rank.items() if node[0] == 'a'}

korean_movies = {movie:weight for movie, weight in movie_rank.items() if '한국)' in idx2movie(movie.split()[1])}

# movies starring many famous actors
for movie, _ in sorted(korean_movies.items(), key=lambda x:-x[1])[:30]:
    if movie in history_set:
        continue
    movie_idx = movie.split()[1]    
    print(idx2movie(movie_idx), idx2year(movie_idx))

깡패 수업 2 (1999, 한국) 1999
헬로우 닥터 (한국) 0
작은 연못 (2010, 한국) 2010
4등 (2016, 한국) 2016
세로본능 (한국) 0
수상한 가정부 (2013, 한국) 2013
라라 선샤인 (2009, 한국) 2009
바다 (2011, 한국) 2011
드라마 스페셜 - 붉은 달 (2015, 한국) 2015
악인은 살아 있다 (2015, 한국) 2015
탐정 : 더 비기닝 (2015, 한국) 2015
거북이 달린다 (2009, 한국) 2009
청출어람 (2012, 한국) 2012
착하지 않은 여자들 (2015, 한국) 2015
특종: 량첸살인기 (2015, 한국) 2015
인수대비 (2011, 한국) 2011
오빠생각 (2016, 한국) 2016
아티스트 (2000, 한국) 2000
형 (2016, 한국) 2016
미생 (2014, 한국) 2014
1999, 면회 (2013, 한국) 2013
이 공을 받아줘 (한국) 0
미생 프리퀄 (2013, 한국) 2013
원라인 (한국) 0
기타가 웃는다 (한국) 0


In [15]:
# top actor
for actor, _ in sorted(actor_rank.items(), key=lambda x:-x[1])[:100]:
    actor_idx = actor.split()[1]
    print(idx2actor(actor_idx), end=' / ')

Wilhelmina McFadden / Tyler John Williams / 채상우 / Unknown / Unknown / Unknown / Unknown / Unknown / 이광일 / 차은재 / 강성해 / Song Kang-ho / 이항나 / Siwan / 조정석 / 이종석 / 이사라 / 전혜진 / Steve Park / 김의건 / 이찬영 / Adnan Haskovic / 엄홍식 / Moon Geun-Young / 허형규 / 진지희 / Igor Juric / 곽병규 / 권수현 / 최장원 / 조완기 / Baek Yoon-Sik / 서예지 / 정원중 / Clark Middleton / 이정재 / 김의성 / 김무열 / 박소담 / 전효정 / 이세연 / 오달수 / 이도엽 / 심희섭 / Unknown / 유재상 / 최민철 / 정가람 / 조영규 / 고준 / 윤혜경 / 정규수 / 차광수 / 김지나 / 최재진 / 지이수 / 송영창 / Unknown / 김대명 / Unknown / Emma Levie / Chris Evans / 김해숙 / 박원상 / 남다름 / 강지우 / 최성원 / Joseph Hieu / 백인권 / 이성민 / D.O. / Martti Suosalo / 유상재 / 조민기 / 김영애 / 박정민 / 오혜지 / 김명식 / 민수현 / 황도연 / Luke Pasqualino / 김강산 / Ko Asung / 김보성 / Octavia Spencer / 서현정 / 이수인 / 박해준 / 곽자형 / Pietro Castellitto / Jamie Bell / 한기중 / 박현지 / 이동희 / 낸시 / 박기산 / 김동영 / Kenny Doughty / 김보정 / Alison Pill / 